In [ ]:
import torch
import numpy as np
import trimesh
from matplotlib import pyplot as plt
import json
from utils.uv_tools import apply_displacement
from data.garment_dataset import GarmentDataset
from data.amass_dataset import AmassDataset
from utils.file_management import list_files
from data.normalization import unnormalize_cloth
from tqdm.contrib.concurrent import process_map  # or thread_map
from utils.visualization import render, animation, imshow, plot_dict

In [ ]:
with open("../configs/config.json") as f:
    config = json.load(f)
dataset = GarmentDataset(config, device="cuda:0")
body_model = dataset.body_model
template = trimesh.Trimesh(dataset.template.v.cpu(), dataset.template.f.cpu(), vertex_colors=np.full_like(dataset.template.v.cpu(), (0.8,0.8,0.8)))

In [ ]:
print(len(dataset))

In [ ]:
bending = [d['bending'] for d in dataset.lazy_data]
stretching = [d['stretching'] for d in dataset.lazy_data]
density = [d['density'] for d in dataset.lazy_data]

In [ ]:
bending.sort()
stretching.sort()
density.sort()

In [ ]:
plt.hist(density, np.linspace(density[0], density[-1], 50), histtype='bar', rwidth=0.8)
plt.show()

In [ ]:
plt.hist(bending, np.linspace((bending[0]), (bending[-1]), 50), histtype='bar', color='orange')
plt.hist(bending, np.logspace(np.log10(bending[0]), np.log10(bending[-1]), 50), histtype='step')

plt.show() # should be uniform

In [ ]:
plt.hist(stretching, np.linspace(stretching[0], stretching[-1], 50), histtype='bar', rwidth=0.8)
plt.show()

In [ ]:
vertices = dataset[0]['cloth_vertices'].cpu()
print(vertices)
ground_truth = trimesh.Trimesh(vertices, dataset.template.f.cpu(), vertex_colors=np.full_like(dataset.template.v.cpu(), (1,0,0)))
# v = animation([ground_truth, template])

In [ ]:
displaced = apply_displacement(dataset.template, dataset[0]["vdm"], dataset.img_size)

displacement = displaced - dataset[0]["cloth_vertices"]
norm = torch.linalg.vector_norm(displacement, dim=1)
sort_norm = torch.argsort(norm)

color = torch.tile(torch.tensor([[1.,0.,0.,1.]]), (norm.shape[0], 1))
color [:,0] = color [:,0] * norm.cpu() * 1000
color [color > 1.] = 1.
color [color < 0.] = 0.

gt = trimesh.Trimesh(dataset[0]['cloth_vertices'].cpu(), dataset.template.f.cpu(), vertex_colors=np.full_like(color, 1))
mesh = trimesh.Trimesh(displaced.cpu(), dataset.template.f.cpu(), vertex_colors=color)
template = trimesh.Trimesh(dataset.template.v.cpu(), dataset.template.f.cpu(), vertex_colors=np.full_like(color, 1))

print(norm.max())
print(norm.mean())
print(norm.shape)
print(dataset[0]["vdm"].shape)
# (gt+mesh+template).show()
mesh.show()

In [ ]:
sequences = dataset.get_seq_dict()

In [ ]:
print(len(sequences.keys()))

In [ ]:
sequences.keys()

In [ ]:
seq = sequences['/home/adumouli/Data/MY_DATASET/Dataset/accad/walk/B1-standtowalk_poses/Cos/data0/output']
SEQ_START = seq[0]
SEQ_LEN = seq[1]

In [ ]:
meshes = []
datas = []
conditions = []
for i in range(SEQ_START, SEQ_START+SEQ_LEN, 10):
    data = dataset[i]
    datas.append(data)

body_faces = body_model.get_faces().cpu()
body_sequence = []
ground_truth = []
for i, data in enumerate(datas):
    body_vertices = body_model(data)[-1].cpu()
    body_sequence.append(trimesh.Trimesh(body_vertices, body_faces, vertex_colors=np.full_like(body_vertices, (1,1,1)), process=False))

    data_cp = data.copy()
    unnormalize_cloth(data_cp)
    v = data_cp['cloth_vertices'].cpu()
    ground_truth.append(trimesh.Trimesh(v, dataset.template.f.cpu(), vertex_colors=np.full_like(v, (1,1,1))*.5))

print(len(body_sequence))

In [ ]:
scene = trimesh.Scene()
for i in range(0, len(body_sequence), 3):
    scene.add_geometry(body_sequence[i] + ground_truth[i])
    # trimesh.Trimesh().export("/home/adumouli/Bureau/sequence/full/frame_" + str(i) + ".obj")
    # ground_truth[i].export("/home/adumouli/Bureau/sequence/ground_truth/frame_" + str(i) + ".obj")
    # body_sequence[i].export("/home/adumouli/Bureau/sequence/body/frame_" + str(i) + ".obj")
scene.show(viewer='gl')

In [ ]:
meshes = []
for i in range(len(body_sequence)):
    frame = body_sequence[i]
    frame +=  ground_truth[i]
    meshes.append(frame)

animation(meshes)

# Body self-intersections

In [ ]:
import torch
import numpy as np
import trimesh
from matplotlib import pyplot as plt
import json
from utils.uv_tools import apply_displacement
from data.garment_dataset import GarmentDataset
from data.amass_dataset import AmassDataset
from utils.file_management import list_files
from data.normalization import unnormalize_cloth
from tqdm.contrib.concurrent import process_map  # or thread_map
from utils.visualization import render, Sequencevisualizer, imshow, plot_dict, save_video

import time
import argparse
import torch
import numpy as np
import json
import pickle
import re

import trimesh
import pyrender
from mesh_intersection.bvh_search_tree import BVH
from utils.file_management import list_files
from data.smpl_sequence import SMPL_Sequence
from body_models.smpl_model import SMPLH
from tqdm.auto import tqdm
from utils.geometry import l2dist

In [ ]:
# # compute smpl edge lengthes

# zero_smpl = trimesh.load_mesh('/home/adumouli/Bureau/garment-diffusion/src/zero_smpl.obj')
# # zero_smpl = zero_smpl.subdivide_loop(2)

# color = torch.Tensor(zero_smpl.vertices)
# length = torch.Tensor(zero_smpl.edges_unique_length)
# color[zero_smpl.edges_unique] = length
# trimesh.Trimesh(zero_smpl.vertices, zero_smpl.faces, vertex_colors=color).show(viewer='gl')

# # color = torch.Tensor(zero_smpl.area_faces)[..., None].expand(-1,4).clone()
# # color[:,0] *= 1e3
# # color[:,1:3] = 0
# # color[:,3] = 1
# # trimesh.Trimesh(zero_smpl.vertices, zero_smpl.faces, face_colors=color).show(viewer='gl')

# # plt.boxplot(zero_smpl.edges_unique_length);
# # plt.boxplot(1/zero_smpl.area_faces);

In [ ]:
with open("/home/adumouli/Bureau/garment-diffusion/configs/config.json") as f:
    config = json.load(f)
amass = AmassDataset(config)

fixed_amass = AmassDataset(config, config["dataset"]["folder"])

In [ ]:
for i, seq in enumerate(amass):
    if re.search("leap", seq.npz_file) is not None:
        print(i)


In [ ]:
# compute smpl native self-intersections

device = 'cuda'
m = BVH(max_collisions=8)

# zero_smpl = trimesh.load_mesh('/home/adumouli/Bureau/garment-diffusion/src/zero_smpl.obj')
# vertices = zero_smpl.vertices

data = amass[0].data
data['poses'] = torch.zeros_like(data['poses'])
data['betas'] = torch.zeros_like(data['betas'])

vertices = amass.body_model(data)[0]
faces = amass.body_model.get_faces().cpu()
triangle_count = float(faces.shape[0])

triangles = vertices[faces]
triangles = torch.Tensor(triangles).to(device)[None,...]

outputs = m(triangles)
mask = outputs[..., 0] >= 0
collisions = outputs[mask]

count_before = torch.count_nonzero(mask, dim=-1) * (100 / triangle_count)

zero_intersections = count_before.detach().cpu()[0].item()

print(zero_intersections)

In [ ]:
import pickle
with open("../log/amass_intersect_fix_30.pkl", 'rb') as f:
    previous = pickle.load(f)

with open("../log/amass_intersect_fix_snug.pkl", 'rb') as f:
    result = pickle.load(f)

previous_table = []
for k in previous.keys():
    previous_table.append(previous[k].mean())

table = []
for k in result.keys():
    table.append(result[k].mean())

fig,(ax) = plt.subplots(figsize=(4,4), ncols=1)
# ax.set_title('Self-intersections distribution in selected AMASS sequences')
ax.set_xlabel('Sequence')
ax.set_ylabel('Intersecting face percentage')
ax.plot(table, label='fix')
ax.plot(previous_table, label='original')
plt.axhline(y = zero_intersections, color = 'r', linestyle = '-', label='zero')
ax.legend()

fig,(ax) = plt.subplots(figsize=(4,4), ncols=1)
# ax.set_title('Self-intersections distribution in selected AMASS sequences')
ax.set_xlabel('Intersecting face percentage')
ax.set_ylabel('Sequence count')
ax.hist(table, alpha=0.5, label='fix')
ax.hist(previous_table, alpha=0.5, label='original')
plt.axvline(x = zero_intersections, color = 'r', linestyle = '-', label='zero')
ax.legend()

fig,(ax) = plt.subplots(figsize=(2,2), ncols=2)
# ax.set_ylabel('Intersecting face percentage')
ax[0].boxplot(previous_table)
ax[0].set_ylabel('Intersecting face %')
ax[0].set_xlabel('orig')
ax[1].boxplot(table)
ax[1].set_xlabel('fix')
ax[1].get_yaxis().set_visible(False)

In [ ]:
import pickle
with open("../log/amass_fix_log_before.pkl", 'rb') as f:
    previous = pickle.load(f)

with open("../log/amass_fix_log_new.pkl", 'rb') as f:
    result = pickle.load(f)

previous_table = []
for k in previous.keys():
    previous_table.append(previous[k].mean())

table = []
for k in result.keys():
    table.append(result[k].mean())

fig,(ax) = plt.subplots(figsize=(4,4), ncols=1)
# ax.set_title('Self-intersections distribution in selected AMASS sequences')
ax.set_xlabel('Sequence')
ax.set_ylabel('Intersecting face percentage')
ax.plot(table, label='fix')
ax.plot(previous_table, label='original')
plt.axhline(y = zero_intersections, color = 'r', linestyle = '-', label='zero')
ax.legend()

fig,(ax) = plt.subplots(figsize=(4,4), ncols=1)
# ax.set_title('Self-intersections distribution in selected AMASS sequences')
ax.set_xlabel('Intersecting face percentage')
ax.set_ylabel('Sequence count')
ax.hist(table, alpha=0.5, label='fix')
ax.hist(previous_table, alpha=0.5, label='original')
plt.axvline(x = zero_intersections, color = 'r', linestyle = '-', label='zero')
ax.legend()

fig,(ax) = plt.subplots(figsize=(2,2), ncols=2)
# ax.set_ylabel('Intersecting face percentage')
ax[0].boxplot(previous_table)
ax[0].set_ylabel('Intersecting face %')
ax[0].set_xlabel('orig')
ax[1].boxplot(table)
ax[1].set_xlabel('fix')
ax[1].get_yaxis().set_visible(False)

In [ ]:
plot_dict(result, legend=False)

In [ ]:
import pickle

with open("../log/amass_intersect_log.pkl", 'rb') as f:
    result = pickle.load(f)
with open("../log/amass_intersect_fix_snug.pkl", 'rb') as f:
    new_result = pickle.load(f)
# Create a list of tuples: (key, max_value, mean_value, frame_of_max)
stats = []
for k in result.keys():
    max_val = result[k].max()
    mean_val = result[k].mean()
    frame_of_max = result[k].argmax()
    diff = (result[k] - new_result[k]).min()
    diff_idx = (result[k] - new_result[k]).argmin()
    stats.append((k, max_val, mean_val, frame_of_max, diff, diff_idx))

# Sort the list based on max_value in descending order
sorted_stats = sorted(stats, key=lambda x: x[4], reverse=True)

# Print the sorted keys and corresponding max values
for k, max_val, mean_val, frame_of_max, diff, diff_idx in sorted_stats:
    print(f"Key: {k}, Max: {max_val:.4f}, Mean: {mean_val:.4f}, Frame: {frame_of_max}, Diff: {diff}:{diff_idx}")

In [ ]:
out = "/home/adumouli/Data/MY_DATASET/SELECTED_AMASS"
file_list = list_files(out, pattern="poses.npz")
device = 'cuda'
faces = amass.body_model.get_faces().cpu()
number = 0
key = sorted_stats[number][0]

key = "Running_c3d/C19"
for i, f in enumerate(file_list):
    if re.search(key, f) is None:
        continue
    
    print(f)
    seq = SMPL_Sequence(f, device=device).to(device)
    data = seq.data
    data["poses"] = data["poses"][:, :66]
    data["betas"][1] = -2
    verts = amass.body_model(data).cpu()
    meshes = [trimesh.Trimesh(v, faces) for v in verts]

    from utils.smpl_tools import separate_arms
    data["poses"] = separate_arms(data["poses"], 20)
    verts = amass.body_model(data).cpu()
    for idx, v in enumerate(verts):

        v[:,2] += 2
        meshes[idx] += trimesh.Trimesh(v, faces)

    viewer = Sequencevisualizer(meshes)
    print(meshes)
    break

In [ ]:
fig,(ax) = plt.subplots(figsize=(4,4), ncols=1)
ax.set_title('KIT/348/walking_fast06')
ax.set_xlabel('Frame')
ax.set_ylabel('Intersecting face percentage')
plt.plot(result[key])
plt.axhline(y = 0.73, color = 'r', linestyle = '-')
fig.show()

In [ ]:
plt.plot(mean, label='mean')
plt.plot(maximum, label='max')
plt.legend()

In [ ]:
from body_models.smpl_model import SMPLH
from data.smpl_sequence import SMPL_Sequence
import json
import torch
import trimesh
import pickle
with open("./amass_fix_log_new.pkl", 'rb') as f:
    result = pickle.load(f)

tmp = 0
key = None
for k in result.keys():
    if result[k].max() > tmp:
        tmp = result[k].max()
        key = k

with open("/home/adumouli/Bureau/garment-diffusion/configs/config.json") as f:
    config = json.load(f)
    config["dtype"] = torch.float32
    # config["device"] = 'cpu'
    smplh = SMPLH(config)
faces = smplh.get_faces().to("cpu", torch.long)

seq = SMPL_Sequence(key)
data = dict()
data["trans"] = seq.data["trans"].to("cuda", dtype=torch.float32)
data["poses"] = seq.data["poses"][:, :66].to("cuda", dtype=torch.float32)
data["betas"] = seq.data["betas"][None,...].to("cuda", dtype=torch.float32)
vertices = smplh(data).to(dtype=torch.float32).detach().cpu()

sequence = [trimesh.Trimesh(i, faces) for i in vertices]

In [ ]:
data = amass[0].data

vertices = smplh(data).to(dtype=torch.float32).detach().cpu()
sequence = [trimesh.Trimesh(i, faces) for i in vertices]

In [ ]:
from tqdm.auto import tqdm
import os
import tempfile
from PIL import Image
from itertools import repeat
import trimesh
import pyrender
from tqdm.contrib.concurrent import process_map  # or thread_map
import numpy as np

def _renderSeq(meshes: list[trimesh.Trimesh], folder):
    renderer = pyrender.OffscreenRenderer(512, 512)
    meshes = process_map(pyrender.Mesh.from_trimesh, meshes, max_workers=8)

    table=[]

    camera = pyrender.PerspectiveCamera(yfov=np.pi / 3.0, aspectRatio=1.414)
    # camera = pyrender.OrthographicCamera(xmag=5.0, ymag=5.0)
    light = pyrender.DirectionalLight(color=[1.0, 1.0, 1.0], intensity=2.0)
    # grid = pyrender.Mesh.from_trimesh(trimesh.creation.box(extents=[20, 20, 0.01]))
    # grid = pyrender.Mesh.from_points([[20,20,0], [20,-20,0],[-20,-20,0],[-20,20,0]], colors=[1,0,0]*4)
    trimesh.transformations.rotation_matrix(np.pi/2, [0,0,1])
    x=-np.pi/4
    cam_pose = np.array([
    [1.0, 0.0, 0.0, -1],
    [0.0, np.cos(x), np.sin(x), -2],
    [0.0, -np.sin(x), np.cos(x), 3],
    [0.0, 0.0, 0.0, 1.0]
    ])
    for i, mesh in enumerate(meshes):
        scene = pyrender.Scene(ambient_light=[0.02, 0.02, 0.02], bg_color=[1.0, 1.0, 1.0])
        # cam_pose = trimesh.transformations.translation_matrix(mesh.centroid*[1, 1, 0]+[0,0,2])
        scene.add(mesh, pose=np.eye(4))
        scene.add(camera, pose=cam_pose)
        scene.add(light, pose=np.eye(4))
        arr, _ = renderer.render(scene)
        img = Image.fromarray(arr)
        img.save(os.path.join(folder, f"img{i}.png"))

    renderer.delete()

def save_video(sequence: list[trimesh.Trimesh], outputfile="./out.mp4", fps=50):
    with tempfile.TemporaryDirectory() as tmp_dir:
        _renderSeq(sequence, tmp_dir)
        os.system(f"ffmpeg -r {fps} -i {tmp_dir}/img%01d.png -b 5000k -vcodec h264 -y {outputfile}")

# save_video(sequence)

In [ ]:
import torch
import numpy as np
import trimesh
from matplotlib import pyplot as plt
import json
from data.garment_dataset import GarmentDataset
from data.amass_dataset import AmassDataset
from sklearn.manifold import TSNE
from tqdm.auto import tqdm

with open("/home/adumouli/Bureau/garment-diffusion/configs/config.json") as f:
    config = json.load(f)
dataset = AmassDataset(config, device='cpu')

In [ ]:
key = 'poses' # change in ['betas', 'poses', 'trans']

color = torch.empty(0)
poses = np.empty((0, 66))
body_verts = torch.empty(0)
for seq in tqdm(range(len(dataset))):
    data = dataset[seq].sequence_data(25)
    # data = dataset[seq].data

    # values = aa2six(data[key].reshape(data[key].shape[0],-1,3)).flatten(1,2)
    values = data[key]
    values = np.unwrap(values.cpu().numpy())
    poses = np.append(poses, values, axis=0)

    vertices = dataset.body_model(data)
    body_verts = torch.cat((body_verts, vertices))

    color = torch.cat((color, torch.rand((1, 3)).expand(values.shape[0], 3)))
    
color = color.cpu().numpy()
body_verts = body_verts.cpu().flatten(1,2).numpy()

In [ ]:
print(poses.shape)

In [ ]:
plt.scatter(body_verts[...,0], body_verts[...,1], s=10, c=color, alpha=0.80)

In [ ]:
plt.scatter(poses[...,0], poses[...,1], s=10, c=color, alpha=0.80)

In [ ]:
tsne = TSNE(verbose=True)
poses_tsne = tsne.fit_transform(poses)

In [ ]:
tsne = TSNE(verbose=True)
body_verts_tsne = tsne.fit_transform(body_verts)

In [ ]:
def visualize_tsne(data, target=None):
    """
    Visualize t-SNE 2 dimnetinal graph and use stratified target as the color

    Parameters:
    data: Input Array containing the output of T-SNE transformation
    target: Input Array or Dataframe containing the stratified Target
    
    Returns:
    t-SNE graph
    """
    plt.figure(figsize=(5, 5))
    plt.scatter(data[:,0], data[:,1],
            c=target, 
            edgecolor='none', 
            alpha=0.80, 
            s=10)
    plt.axis('off')
    plt.show()

In [ ]:
visualize_tsne(body_verts_tsne, color)

In [ ]:
visualize_tsne(poses_tsne, color)

In [ ]:
key = 'poses' # change in ['betas', 'poses', 'trans']

train_p = torch.empty(0)
for seq in tqdm(range(3)):
    # values = data[key]
    train_p = torch.cat((train_p, values))
    
train_p = train_p.cpu().flatten(1,2).numpy()

In [ ]:
def show_sequence(vertices, faces, frame_interval=40):
    scene = trimesh.Scene()
    for i in list(range(0, vertices.shape[0]-1, frame_interval)) + [vertices.shape[0]-1]:
        mesh = trimesh.Trimesh(vertices=vertices[i, :, :].detach().cpu(), faces=faces, process=False)
        scene.add_geometry(mesh)
    return display(scene.show())

In [ ]:
data = dataset[0].data
values = dataset.body_model(data)
show_sequence(values.cpu(), dataset.smpl_faces.cpu())

In [ ]:
data = dataset[1].data
values = dataset.body_model(data)
show_sequence(values.cpu(), dataset.smpl_faces.cpu())

In [ ]:
data = dataset[2].data
values = dataset.body_model(data)
show_sequence(values.cpu(), dataset.smpl_faces.cpu())